#  Recomendador de pelis

![](http://labs.criteo.com/wp-content/uploads/2017/08/CustomersWhoBought3.jpg)

En este cuaderno, se busca  implementar algunos algoritmos de recomendación (basados en contenido, en popularidad y en filtrado colaborativo) y se va a construir un conjunto (ensemble) de estos modelos para crear nuestro sistema de recomendación final. Contamos con dos conjuntos de datos de MovieLens.

El conjunto de datos completo: Consiste en 26,000,000 de calificaciones y 750,000 aplicaciones de etiquetas aplicadas a 45,000 películas por 270,000 usuarios. Incluye datos del genoma de etiquetas con 12 millones de puntuaciones de relevancia en 1,100 etiquetas.

El conjunto de datos pequeño: Comprende 100,000 calificaciones y 1,300 aplicaciones de etiquetas aplicadas a 9,000 películas por 700 usuarios.

Construye un **Recomendador Simple** utilizando películas del conjunto de datos completo, mientras que todos los sistemas de recomendación personalizados harán uso del conjunto de datos pequeño (debido a que mi capacidad de cómputo es muy limitada). Como primer paso, construiré mi sistema de recomendación simple.

In [86]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
##from surprise import Reader, Dataset, SVD
##from surprise.model_selection import cross_validate

import warnings
warnings.simplefilter('ignore')




### Análisis de datos y manipulación
import pandas as pd  # Estructuras de datos flexibles y herramientas de análisis (DataFrames, Series)

import numpy as np   # Computación numérica con arrays multidimensionales y funciones matemáticas

### Visualización de datos
import matplotlib.pyplot as plt  # Biblioteca fundamental para crear visualizaciones estáticas, animadas e interactivas

import seaborn as sns            # Biblioteca de visualización basada en matplotlib con estilos atractivos y gráficos estadísticos

### Análisis estadístico
from scipy import stats  # Funciones estadísticas (distribuciones, tests estadísticos, medidas de correlación)

### Evaluación segura de strings
from ast import literal_eval  # Convierte strings que contienen estructuras de Python en las estructuras mismas

### Procesamiento de texto y machine learning
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  # Conversión de texto a vectores numéricos

from sklearn.metrics.pairwise import linear_kernel, cosine_similarity  # Cálculo de similitudes entre vectores

### Procesamiento de lenguaje natural (NLP)
from nltk.stem.snowball import SnowballStemmer  # Algoritmo de stemming para reducir palabras a su raíz

from nltk.stem.wordnet import WordNetLemmatizer # Lematización para reducir palabras a su forma canónica

from nltk.corpus import wordnet                 # Base de datos léxica del inglés para lematización

### Sistema de recomendación
from surprise import Reader, Dataset, SVD       # Biblioteca para sistemas de recomendación (Reader: formato de datos, Dataset: conjunto de datos, SVD: Descomposición en Valores Singulares)

from surprise.model_selection import cross_validate  # Validación cruzada para modelos de recomendación


## Recomendador Simple 

El Recomendador Simple ofrece recomendaciones generales a todos los usuarios basadas en la popularidad de las películas y (a veces) en su género. La idea básica detrás de este recomendador es que las películas más populares y con mejores críticas tienen una mayor probabilidad de ser del agrado del público promedio. Este modelo no proporciona recomendaciones personalizadas basadas en el usuario (como nuestro reto).

La implementación de este modelo es extremadamente sencilla. Todo lo que debemos hacer es ordenar las películas según sus calificaciones y popularidad, y mostrar las películas principales de la lista. Como paso adicional, podemos incluir un argumento de género para obtener las mejores películas de un género en particular.

Cargar los datos

In [87]:
md = pd.read_csv('movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Vamos a trabajar por género

In [88]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

Usaremos las calificaciones de TMDb para crear nuestro Ranking de las Mejores Películas (Top Movies Chart). La fórmula de calificación ponderada de IMDB para construir este ranking. Matemáticamente, se representa de la siguiente manera:

Calificación Ponderada (WR) = $(\frac{v}{v + m} \cdot R) + (\frac{m}{v + m} \cdot C)$

donde,

v es el número de votos que tiene la película

m es el número mínimo de votos requeridos para ser incluida en el ranking

R es la calificación promedio de la película

C es la calificación promedio de todas las películas del conjunto de datos

El siguiente paso es determinar un valor apropiado para m, el número mínimo de votos necesarios para aparecer en el ranking. Usaremos el percentil 95 como punto de corte. En otras palabras, para que una película aparezca en el ranking, debe tener más votos que al menos el 95% de las películas en la lista.

El ranking general de las 250 mejores películas se define una función para crear rankings por género específico. 

In [89]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

np.float64(5.244896612406511)

In [90]:
m = vote_counts.quantile(q = .95)# completa la línea
m

np.float64(434.0)

Extracción del año:

In [91]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

Vamos a trabajar con el siguiente data frame:

In [92]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

Por lo tanto, para calificar y ser considerada en el ranking, una película debe tener al menos *Completa* votos en TMDB. También observamos que la calificación promedio de una película en TMDB es de *Completa en una escala de 10. En total, *completa* películas califican para estar en nuestro ranking.

In [93]:
def weighted_rating(x):
    #Función de Calificación ponderada
    v = x['vote_count']
    R = x['vote_average']
    
    
    return (v/(v+m))*R+(m/(v+m))*C

In [94]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [95]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

### Películas Top 

In [96]:
qualified.head(n = 15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


Vemos que tres películas de Christopher Nolan —Inception, The Dark Knight e Interstellar, aparecen en los primeros lugares de nuestro ranking. El gráfico también muestra una fuerte preferencia de los usuarios de TMDB hacia ciertos géneros y directores.

Ahora construiremos una función que genere rankings para géneros específicos. Para ello, relajaremos nuestras condiciones por defecto utilizando el percentil 85 en lugar del 95.

In [97]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [98]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

Veamos el método mostrando el Top 15 de películas de romance (el género de romance casi no apareció en nuestro ranking general, a pesar de ser uno de los géneros cinematográficos más populares).

### Top 15 de películas de Romance 

In [99]:
build_chart(genre="Romance").head(n = 15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


La mejor película de romance según nuestras métricas es la producción de Bollywood Dilwale Dulhania Le Jayenge. Esta cinta protagonizada por Shahrukh Khan.

## Recomendador Basado en Contenido

El recomendador que construimos en la sección anterior presenta algunas limitaciones importantes. Por ejemplo, ofrece las mismas recomendaciones a todos los usuarios, sin tener en cuenta sus gustos personales. Si una persona que ama las películas románticas (y detesta la acción) revisará nuestro Top 15, probablemente no disfrutaría de la mayoría de las películas. Incluso si revisara nuestros rankings por género, aún no estaría recibiendo las mejores recomendaciones.

Por ejemplo, consideremos a una persona que ama *Dilwale Dulhania Le Jayenge*, *My Name is Khan* y *Kabhi Khushi Kabhi Gham*. Podemos inferir que esta persona disfruta de las películas protagonizadas por el actor **Shahrukh Khan** y dirigidas por **Karan Johar**. Aun si consultara el ranking de películas románticas, probablemente no encontraría estas entre las principales recomendaciones.

Para personalizar mejor nuestras recomendaciones, hay que construir un motor que calcule la **similitud entre películas** basándose en ciertos atributos, y que sugiera aquellas más parecidas a una película que le haya gustado al usuario. Dado que utilizaremos metadatos de las películas (o su contenido) para construir este motor, este enfoque se conoce como **Filtrado Basado en Contenido**.

Vamops a construir dos recomendadores basados en contenido, a partir de:
* Sinopsis y eslóganes de las películas  
* Reparto, equipo técnico, palabras clave y género  

Como se mencionó en la introducción, se utiliza un subconjunto de todas las películas disponibles debido a las limitaciones de capacidad de cómputo.


In [100]:
links_small = pd.read_csv('links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [101]:
md = md.drop([19730, 29503, 35587])

In [102]:
md['id'] = md['id'].astype('int')

In [103]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

 Tenemos **Completa** películas disponibles en nuestro conjunto de datos reducido de metadatos de películas,
 el cual es **Completa** veces más pequeño que nuestro conjunto de datos original de 45,000 películas.


### Recomendador Basado en la Descripción de la Película

Primero intentemos construir un recomendador utilizando las **descripciones** y **eslóganes** de las películas.  
No contamos con una métrica cuantitativa para evaluar el rendimiento de nuestro modelo, por lo que esta evaluación deberá realizarse de forma **cualitativa**.


In [104]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

Se analiza las palabras en la descripción, creando unigramas y bigramas

In [105]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [106]:
tfidf_matrix.shape

(9099, 268124)

#### Similitud del Coseno

Vamos a utilizar la **Similitud del Coseno** para calcular una cantidad numérica que indique el grado de similitud entre dos películas.  
Matemáticamente, se define de la siguiente manera:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Dado que hemos utilizado el **Vectorizador TF-IDF**, calcular el **producto punto** nos dará directamente el valor de la similitud del coseno.  
Por lo tanto, utilizaremos la función **linear_kernel** de *sklearn* en lugar de *cosine_similarities*, ya que es más rápida.


In [107]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [108]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ], shape=(9099,))

Ahora tenemos una **matriz de similitud del coseno por pares** para todas las películas de nuestro conjunto de datos.  
El siguiente paso es escribir una función que devuelva las **30 películas más similares** basándose en el puntaje de similitud del coseno.


In [109]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [110]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Ahora intentemos obtener las **principales recomendaciones** para algunas películas y ver qué tan buenas son las sugerencias.


In [111]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [112]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

Podemos observar que para **The Dark Knight**, nuestro sistema logra identificarla como una película de *Batman* y, en consecuencia, recomienda otras películas de *Batman* como sus principales sugerencias.  
Sin embargo, lamentablemente eso es todo lo que este sistema puede hacer por el momento.  
Esto no resulta muy útil para la mayoría de las personas, ya que no toma en cuenta características muy importantes como el **elenco**, **equipo técnico**, **director** y **género**, los cuales influyen en la calificación y popularidad de una película.  
Alguien a quien le gustó **The Dark Knight** probablemente la disfruta más por **Nolan**, y seguramente detestaría **Batman Forever** y otras películas de menor calidad dentro de la franquicia de Batman.

Por lo tanto, pueden estudiar los  metadatos,  que pueden ser más representativos que solo la **sinopsis** y el **eslogan**.  
En la siguiente subsección construiremos un recomendador más sofisticado que tenga en cuenta el **género**, las **palabras clave**, el **reparto** y el **equipo técnico**.


### Recomendador Basado en Metadatos

Para construir nuestro recomendador estándar basado en contenido y metadatos, necesitaremos combinar nuestro conjunto de datos actual con los conjuntos de datos de **equipo técnico (crew)** y **palabras clave (keywords)**.  
Prepararemos estos datos como **primer paso**.


In [113]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

In [114]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [115]:
md.shape

(45463, 25)

In [116]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [117]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

Ahora tenemos el **reparto**, el **equipo técnico**, los **géneros** y los **créditos**, todos en un mismo *dataframe*.  
Vamos a procesar estos datos un poco más utilizando las siguientes intuiciones:

1. **Equipo técnico (Crew):**  
   Del equipo técnico, solo tomaremos al **director** como característica, ya que los demás miembros no influyen tanto en la *esencia* de la película.

2. **Reparto (Cast):**  
   La selección del reparto es un poco más complicada.  
   Los actores poco conocidos o los papeles menores no afectan demasiado la opinión del público sobre una película.  
   Por lo tanto, debemos seleccionar solo los **personajes principales** y sus respectivos actores.  
   De manera arbitraria, elegiremos a los **3 actores principales** que aparecen en la lista de créditos.


In [118]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [119]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [120]:
smd['director'] = smd['crew'].apply(get_director)

In [121]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [122]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

El enfoque para construir este recomendador va a ser *experimental*.  
Se planea crear un **volcado de metadatos** (*metadata dump*) para cada película, que contenga los siguientes elementos:  
**géneros, director, actores principales y palabras clave.**  
Luego, se utiliza un **Count Vectorizer** para generar nuestra matriz de conteo, tal como hicimos en el recomendador basado en descripciones.  
Los pasos restantes serán similares a los anteriores: calcularemos las **similitudes del coseno** y devolveremos las películas más parecidas.

Estos son los pasos que sigo para preparar los datos de géneros y créditos:

1. **Eliminar espacios y convertir todo a minúsculas** en todas las características.  
   De esta manera, nuestro motor no confundirá nombres como **Johnny Depp** y **Johnny Galecki**.  

2. **Mencionar al director tres veces** para darle un mayor peso en comparación con el resto del reparto.


In [123]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [124]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [125]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,False,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,"[joejohnston, joejohnston, joejohnston]"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,False,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",7,4,"[howarddeutch, howarddeutch, howarddeutch]"
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,False,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]"
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,False,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",12,7,"[charlesshyer, charlesshyer, charlesshyer]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45985,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[Adventure, Fantasy, Animation, Science Fictio...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,False,5.7,82.0,2001,"[veronicataylor, rachaellillis, maddieblaustein]","[{'credit_id': '52fe45049251416c75048e35', 'de...","[sequel, pokémon, celebi]",4,17,"[kunihikoyuyama, kunihikoyuyama, kunihikoyuyama]"
46424,False,NaN,0,"[Comedy, Drama]",NaN,265189,tt2121382,sv,Turist,"While holidaying in the French Alps, a Swedish...",...,False,6.8,255.0,2014,"[lisalovenkongsli, johannesbahkuhnke, clarawet...","[{'credit_id': '534fd1a80e0a267eb6000e32', 'de...","[female nudity, dark comedy, family vacation, ...",9,15,"[rubenöstlund, rubenöstlund, rubenöstlund]"
46425,False,NaN,0,"[Comedy, Drama]",NaN,265189,tt2121382,sv,Turist,"While holidaying in the French Alps, a Swedish...",...,False,6.8,255.0,2014,"[lisalovenkongsli, johannesbahkuhnke, clarawet...","[{'credit_id': '534fd1a80e0a267eb6000e32', 'de...","[female nudity, dark comedy, family vacation, ...",9,15,"[rubenöstlund, rubenöstlund, rubenöstlund]"
46426,False,NaN,0,"[Comedy, Drama]",NaN,265189,tt2121382,sv,Turist,"While holidaying in the French Alps, a Swedish...",...,False,6.8,255.0,2014,"[lisalovenkongsli, johannesbahkuhnke, clarawet...","[{'credit_id': '534fd1a80e0a267eb6000e32', 'de...","[female nudity, dark comedy, family vacation, ...",9,15,"[rubenöstlund, rubenöstlund, rubenöstlund]"


#### Palabras Clave (Keywords)

Realizaremos una pequeña **preparación previa** de nuestras palabras clave antes de utilizarlas.  
Como primer paso, calcularemos la **frecuencia de aparición** de cada palabra clave que aparece en el conjunto de datos.


In [126]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [127]:
s = s.value_counts()
s[:5]

keyword
independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: count, dtype: int64

Las palabras clave aparecen con frecuencias que van de **1 a 610**.  
No nos sirven aquellas palabras clave que aparecen **solo una vez**, por lo que podemos eliminarlas sin problema.  
Finalmente, convertiremos cada palabra a su **raíz léxica (stem)**, de modo que términos como *Dogs* y *Dog* se consideren equivalentes.


In [128]:
s = s[s > 1]

In [129]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [130]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [131]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [132]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [133]:
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [134]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [135]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

Reutilizaremos la función **get_recommendations** que escribimos anteriormente.  
Dado que nuestras puntuaciones de **similitud del coseno** han cambiado, esperamos obtener resultados diferentes (y probablemente mejores).  
Verifiquemos nuevamente el caso de **The Dark Knight** y veamos qué recomendaciones obtenemos esta vez.


In [ ]:
('The Dark Knight').head(10)

7991         The Dark Knight Rises
6186                 Batman Begins
6587                  The Prestige
2077                     Following
7608                     Inception
4125                      Insomnia
3373                       Memento
8573                  Interstellar
7619    Batman: Under the Red Hood
1122                Batman Returns
Name: title, dtype: object

Podemos estar mucho más satisfechos con los resultados obtenidos esta vez.  
Las recomendaciones parecen haber identificado correctamente otras películas de **Christopher Nolan** (debido al alto peso asignado al director) y las han colocado entre las principales sugerencias.  
Disfruté ver **The Dark Knight**, así como otras películas de la lista, incluyendo **Batman Begins**, **The Prestige** y **The Dark Knight Rises**.

Por supuesto, podemos seguir experimentando con este motor probando diferentes **pesos para nuestras características** (directores, actores, géneros),  
limitando la cantidad de **palabras clave** utilizadas en la mezcla (*soup*),  
ponderando los **géneros según su frecuencia**,  
mostrando solo películas del **mismo idioma**, entre otras posibles mejoras.


Veamos también las recomendaciones para otra película: **Mean Girls**,  
que resulta ser la película favorita de mi novia.



In [137]:
get_recommendations('Mean Girls').head(10)

3311               Head Over Heels
4735                 Freaky Friday
1321              The House of Yes
6245              Just Like Heaven
7865         Mr. Popper's Penguins
7292    Ghosts of Girlfriends Past
6923     The Spiderwick Chronicles
8855                      The DUFF
6662         It's a Boy Girl Thing
7333       I Love You, Beth Cooper
Name: title, dtype: object

#### Popularidad y Calificaciones

Una cosa que notamos acerca de nuestro sistema de recomendación es que sugiere películas **sin tener en cuenta las calificaciones ni la popularidad**.  
Es cierto que **Batman and Robin** comparte muchos personajes con **The Dark Knight**, pero fue una película terrible que no debería recomendarse a nadie.

Por lo tanto, añadiremos un mecanismo para **filtrar las malas películas** y devolver únicamente aquellas que sean **populares** y que hayan tenido una **buena recepción crítica**.

Tomaremos las **25 películas principales** basadas en los puntajes de similitud y calcularemos el número de votos correspondiente al **percentil 60**.  
Luego, usando este valor como \( m \), calcularemos la **calificación ponderada** de cada película utilizando la **fórmula de IMDB**, tal como hicimos en la sección del *Recomendador Simple*.


In [138]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [139]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7608,Inception,14075,8,2010,7.917588
8573,Interstellar,11187,8,2014,7.897107
6587,The Prestige,4510,8,2006,7.758148
3373,Memento,4168,8,2000,7.740175
7991,The Dark Knight Rises,9263,7,2012,6.921448
6186,Batman Begins,7511,7,2005,6.904127
1122,Batman Returns,1706,6,1992,5.846862
132,Batman Forever,1529,5,1995,5.054144
9004,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1252,Batman & Robin,1447,4,1997,4.287233


Veamos también las recomendaciones para **Mean Girls**.


In [140]:
improved_recommendations('Mean Girls')

,title,vote_count,vote_average,year,wr
1539,The Breakfast Club,2189,7,1985,6.709602
390,Dazed and Confused,588,7,1993,6.254682
8855,The DUFF,1372,6,2015,5.818541
3696,The Princess Diaries,1063,6,2001,5.781086
4735,Freaky Friday,919,6,2003,5.757786
6245,Just Like Heaven,595,6,2005,5.681521
6923,The Spiderwick Chronicles,593,6,2008,5.680901
7450,American Pie Presents: The Book of Love,454,5,2009,5.119690
7292,Ghosts of Girlfriends Past,716,5,2009,5.092422
7865,Mr. Popper's Penguins,775,5,2011,5.087912


Desafortunadamente, **Batman and Robin** no desaparece de nuestra lista de recomendaciones.  
Esto probablemente se deba a que tiene una calificación de **4**, que está solo ligeramente por debajo del promedio en TMDB.  
Ciertamente no merece un 4 cuando películas increíbles como **The Dark Knight Rises** apenas alcanzan un **7** (cuestión de gustos claro,jejeje).  
Sin embargo, no hay mucho que podamos hacer al respecto.

Por lo tanto, concluiremos aquí nuestra sección del **Recomendador Basado en Contenido**.
